# Импорт модели

В данном ноутбуке реализован класс Model, который позволяет загрузить и импортировать готовую, предобученную модель для предсказания эмоций, а также запускать процесс предсказания на отдельных изображениях (для этого нужно передать методу predict путь до интересующего изображения).

In [6]:
from tensorflow.keras.preprocessing import image
import gdown
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import backend as K
import numpy as np
import os
import zipfile
# if os.name == 'nt':
#     %cd

C:\Users\user


In [1]:
class Model():
    def __init__(self):
        self.url = 'https://drive.google.com/uc?id=1-HDQxtpZKGuKWQhsiDANyc41SL_CwHdU'
        self.predictor = None
        self.clas_indices = {0:'anger', 1:'contempt', 2:'disgust', 3:'fear',
                             4:'happy', 5:'neutral', 6:'sad', 7:'surprise',
                             8:'uncertain'}
        self.__dowload_and_unzip__()
        self.__import_model__()

    def __dowload_and_unzip__(self):
        if not os.path.exists('model'):
            if not os.path.exists('model.zip'):
                gdown.download(self.url, 'model.zip', False)
            if os.name == 'nt':
                with zipfile.ZipFile('model.zip') as file:
                    file.extractall('.')
            else:
                ! unzip model.zip -d .
                ! rm model.zip

    def __import_model__(self):
        self.predictor = models.load_model('model')

    def __preprocess_input__(self, img):
        if isinstance(img, str):
            img = image.load_img(img, target_size=(224,224))
            x_temp = image.img_to_array(img)
        elif isinstance(img, np.ndarray):
            x_temp = np.copy(img)
            x_temp = tf.image.resize(x_temp, (224,224)).numpy()
        else:
            raise Exception('Не верный тип параметра.\
            Вы можете передать либо путь до .jpg изображения либо его numpy представление')
            
        data_format = K.image_data_format()
        assert data_format in {'channels_last', 'channels_first'}

        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

        return x_temp[None,...]

    
    def predict(self, img):
        img = self.__preprocess_input__(img)
        id = np.argmax(self.predictor(img))
        return self.clas_indices[id]